In [ ]:
#1) Reading file
df = pd.read_csv("bundesliga_filtered.csv")
df = df[df["league"] == "Bundesliga"].copy()

#2) "Strength feature engineering"
hist = df[df["season"] < 2023]
g = hist.groupby(["team", "h_a"])["scored"].mean().reset_index()
pivot = g.pivot(index="team", columns="h_a", values="scored").reset_index()
pivot.rename(columns={"h":"home_strength","a":"away_strength"}, inplace=True)
df = df.merge(pivot, on="team", how="left")
df["home"] = (df["h_a"]=="h").astype(int)
df["strength"] = np.where(df["home"]==1, df["home_strength"], df["away_strength"])

#3) Data split ---
train_data = df[df["season"] < 2022].copy()   # 2013‑2021
valid_data = df[df["season"] == 2022].copy()  # 2022
test_data  = df[df["season"] == 2023].copy()  # 2023



In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, brier_score_loss
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV

# ------- features definition -------
features_full = [
    'xG','xGA','npxG','npxGA',
    'deep','deep_allowed',
    'ppda_coef','oppda_coef',
    'home','strength'
]
features_reduced = ['xG','xGA','strength','ppda_coef','deep']

target = 'result'

# ------- encoding target variable -------
le = LabelEncoder()
y_train = le.fit_transform(train_data[target])
y_valid = le.transform(valid_data[target])
y_test  = le.transform(test_data[target])  

In [ ]:
# --- full and reduced features lists ---
features_full = ['xG','xGA','npxG','npxGA','deep','deep_allowed',
                 'ppda_coef','oppda_coef','home','strength']
features_reduced = ['xG','xGA','strength','ppda_coef','deep']

# --- features matrices ---
X_train_full = train_data[features_full].fillna(0)
X_valid_full = valid_data[features_full].fillna(0)
X_test_full  = test_data[features_full].fillna(0)

X_train_red = train_data[features_reduced].fillna(0)
X_valid_red = valid_data[features_reduced].fillna(0)
X_test_red  = test_data[features_reduced].fillna(0)

# --- encoded target variable ---
le = LabelEncoder()
y_train = le.fit_transform(train_data['result'])
y_valid = le.transform(valid_data['result'])
y_test  = le.transform(test_data['result'])


In [ ]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier, PassiveAggressiveClassifier, Perceptron
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier,
    BaggingClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier
)
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import accuracy_score, f1_score, brier_score_loss
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Configurations
features_reduced = ['xG', 'xGA', 'strength', 'ppda_coef', 'deep']
features_full = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed',
                 'ppda_coef', 'oppda_coef', 'home', 'strength']

X_train_r = train_data[features_reduced].fillna(0)
X_test_r  = test_data[features_reduced].fillna(0)

X_train_f = train_data[features_full].fillna(0)
X_test_f  = test_data[features_full].fillna(0)

lb = LabelBinarizer()
y_test_bin = lb.fit_transform(y_test)

# ==================== Models list ====================
models = {
    "Logistic Regression": LogisticRegression(max_iter=500, multi_class='multinomial'),
    "Ridge Classifier": RidgeClassifier(),
    "PassiveAggressive": PassiveAggressiveClassifier(),
    "Perceptron": Perceptron(),
    "GaussianNB": GaussianNB(),
    "BernoulliNB": BernoulliNB(),
    "KNN (k=5)": KNeighborsClassifier(n_neighbors=5),
    "SVC (linear)": SVC(kernel='linear', probability=True),
    "SVC (rbf)": SVC(kernel='rbf', probability=True),
    "NuSVC": NuSVC(probability=True),
    "DecisionTree": DecisionTreeClassifier(),
    "ExtraTree": ExtraTreeClassifier(),
    "RandomForest": RandomForestClassifier(),
    "ExtraTrees": ExtraTreesClassifier(),
    "GradientBoosting": GradientBoostingClassifier(),
    "HistGradBoost": HistGradientBoostingClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "Bagging": BaggingClassifier(),
    "MLPClassifier": MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300),
    "LDA": LinearDiscriminantAnalysis(),
    "QDA": QuadraticDiscriminantAnalysis(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
    "LightGBM": LGBMClassifier(),
    "CatBoost": CatBoostClassifier(verbose=0)
}

# ==================== Score function ====================
def benchmark_models(X_train, X_test, y_train, y_test, config_name=""):
    results = []
    for name, model in models.items():
        try:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            try:
                y_proba = model.predict_proba(X_test)
            except:
                y_proba = np.ones((len(y_pred), 3)) / 3

            acc = accuracy_score(y_test, y_pred)
            f1  = f1_score(y_test, y_pred, average='macro')
            brier = np.mean([
                brier_score_loss(y_test_bin[:, i], y_proba[:, i])
                for i in range(3)
            ])

            results.append({
                'Model': name,
                'Config': config_name,
                'Accuracy': acc,
                'F1_macro': f1,
                'Brier': brier
            })
        except Exception as e:
            print(f"{name} failed: {e}")
    return pd.DataFrame(results)

# ==================== Here modelling is being run ====================
df_reduced = benchmark_models(X_train_r, X_test_r, y_train, y_test, config_name='Reduced')
df_full    = benchmark_models(X_train_f, X_test_f, y_train, y_test, config_name='Full')

df_all = pd.concat([df_reduced, df_full]).sort_values(by="Accuracy", ascending=False).reset_index(drop=True)
print(df_all)


                  Model   Config  Accuracy  F1_macro     Brier
0                   LDA     Full  0.635621  0.569343  0.163625
1             SVC (rbf)  Reduced  0.632353  0.529288  0.165267
2              AdaBoost  Reduced  0.630719  0.556078  0.209241
3      GradientBoosting     Full  0.630719  0.555783  0.162378
4                   LDA  Reduced  0.629085  0.556790  0.166356
5         MLPClassifier  Reduced  0.629085  0.541441  0.165203
6   Logistic Regression     Full  0.625817  0.527097  0.162869
7              AdaBoost     Full  0.625817  0.550109  0.208955
8          SVC (linear)  Reduced  0.625817  0.534995  0.165531
9                   QDA  Reduced  0.624183  0.574631  0.177996
10        MLPClassifier     Full  0.620915  0.538967  0.158160
11  Logistic Regression  Reduced  0.620915  0.515837  0.165469
12     GradientBoosting  Reduced  0.619281  0.530863  0.164689
13             LightGBM     Full  0.619281  0.552855  0.172346
14         SVC (linear)     Full  0.617647  0.527099  0